# Hierarchical Indexing (MultiIndex)

Sometimes it's useful to have more than one index level for your data. This is called hierarchical indexing, or multi-indexing. It lets you work with higher-dimensional data in a two-dimensional format (like a DataFrame).

Let's import our standard tools first.

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Let's see what a multiply indexed `Series` looks like. We'll represent population data for a few states across two years.

In [2]:
# Create a list of arrays for the multi-index
index = [('California', 2010), ('California', 2020),
         ('Texas', 2010), ('Texas', 2020),
         ('New York', 2010), ('New York', 2020)]
populations = [37253956, 39538223,
               25145561, 29145505,
               19378102, 20201249]
pop = pd.Series(populations, index=pd.MultiIndex.from_tuples(index))
pop

California  2010    37253956
            2020    39538223
Texas       2010    25145561
            2020    29145505
New York    2010    19378102
            2020    20201249
dtype: int64

This `Series` now has a `MultiIndex`. You can see the two levels of the index. We can access elements from it like this:

In [3]:
# Access all data for California
pop['California']

2010    37253956
2020    39538223
dtype: int64

In [4]:
# Access data for California in 2020
pop['California', 2020]

39538223

## Methods of MultiIndex Creation

There are several ways to create a `MultiIndex`. The `pd.MultiIndex.from_tuples()` method we just used is a good one, but here are a few others.

In [5]:
# From a list of arrays
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.047217  0.961854
  2  0.124222  0.368494
b 1  0.791733  0.944024
  2  0.989073  0.750993

You can also give the index levels names.

In [6]:
# Naming the index levels
df.index.names = ['key1', 'key2']
df

data1     data2
key1 key2                    
a    1     0.047217  0.961854
     2     0.124222  0.368494
b    1     0.791733  0.944024
     2     0.989073  0.750993

Another way is to use `pd.MultiIndex.from_product`, which creates a `MultiIndex` from the cartesian product of the arrays you provide.

In [7]:
# Using from_product
index = pd.MultiIndex.from_product([['a', 'b'], [1, 2]],
                                   names=['key1', 'key2'])
df_prod = pd.DataFrame(np.random.rand(4, 2), index=index, columns=['data1', 'data2'])
df_prod

data1     data2
key1 key2                    
a    1     0.424367  0.382970
     2     0.418999  0.811223
b    1     0.362696  0.385270
     2     0.272894  0.274023

## Indexing and Slicing a MultiIndex

Indexing and slicing a `MultiIndex` can be straightforward or a bit tricky, depending on what you want to do.

In [8]:
# Accessing the data for Texas
pop['Texas']

2010    25145561
2020    29145505
dtype: int64

You can also use slicing. This slice gets all data for states from California to New York.

In [9]:
pop = pop.sort_index() # note: index must be sorted before slicing
pop.loc['California':'New York']

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
dtype: int64

For more advanced slicing, it's often better to use an `IndexSlice` object. This makes the slicing syntax more intuitive.

In [10]:
idx = pd.IndexSlice
pop[idx[:, 2020]]

California    39538223
New York      20201249
Texas         29145505
dtype: int64

This slice gets all data for the year 2020 across all states.

## Rearranging Multi-Indices

Sometimes you need to reshape your data. The two most important methods for this are `stack()` and `unstack()`.

`unstack()` converts a level of the index into columns.

In [ ]:
print(pop)
pop_df = pop.unstack()
print(pop_df)

# note: .unstack() takes all unique values in the specific index level (identified either via integer index level or index label) and turns them into columns

# meanwhile, .reset_index() takes an index level and converts it into a single new column with the same name as index level (e.g., there will be duplicate values going down row sections)

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64
                2010      2020
California  37253956  39538223
New York    19378102  20201249
Texas       25145561  29145505


`stack()` is the opposite. It converts columns back into an index level.

In [12]:
pop_df.stack()

California  2010    37253956
            2020    39538223
New York    2010    19378102
            2020    20201249
Texas       2010    25145561
            2020    29145505
dtype: int64

You can also use `set_index` to create a `MultiIndex` from columns in a `DataFrame`.

In [13]:
df_flat = pd.DataFrame({'state': ['California', 'California', 'Texas', 'Texas'],
                        'year': [2010, 2020, 2010, 2020],
                        'pop': [37253956, 39538223, 25145561, 29145505]})
df_flat

,state,year,pop
0,California,2010,37253956
1,California,2020,39538223
2,Texas,2010,25145561
3,Texas,2020,29145505


In [14]:
df_multi = df_flat.set_index(['state', 'year'])
df_multi

pop
state      year          
California 2010  37253956
           2020  39538223
Texas      2010  25145561
           2020  29145505

## Modern Data Aggregations and Operations with Multi-Indices

The `level` parameter in aggregation functions is deprecated. Instead, we use `groupby()` for more explicit and powerful aggregations.

In [15]:
# Create a new DataFrame for this example
index = pd.MultiIndex.from_product([[2020, 2021], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2020 1      29.0  39.0  42.0  35.9  35.0  38.8
     2      30.0  36.3  14.0  37.6  45.0  37.1
2021 1      32.0  37.0  37.0  36.5  45.0  39.1
     2      69.0  36.3  53.0  36.1  44.0  37.2

### Grouping and Aggregating

In [16]:
# Group by subject and get the mean
health_data.groupby(level='subject', axis=1).mean()

type        HR       Temp
subject                
Bob    40.000000  37.150000
Guido  36.500000  36.525000
Sue    42.250000  38.050000

### Slicing and Selecting Data

In [17]:
# Select data for a specific year and visit
health_data.loc[(2020, 2)]

subject  type
Bob       HR      30.0
          Temp    36.3
Guido     HR      14.0
          Temp    37.6
Sue       HR      45.0
          Temp    37.1
Name: (2020, 2), dtype: float64

In [18]:
# Select data for a specific year, visit, and subject
health_data.loc[(2020, 1), ('Guido')]

subject  type
Guido    HR      42.0
         Temp    35.9
Name: (2020, 1), dtype: float64

### Resetting and Setting the Index

In [19]:
# Reset the index to columns
health_data_flat = health_data.stack(level=['subject', 'type']).reset_index(name='value')
health_data_flat

year  visit subject  type  value
0  2020      1     Bob    HR   29.0
1  2020      1     Bob  Temp   39.0
2  2020      1   Guido    HR   42.0
3  2020      1   Guido  Temp   35.9
4  2020      1     Sue    HR   35.0
5  2020      1     Sue  Temp   38.8
6  2020      2     Bob    HR   30.0
7  2020      2     Bob  Temp   36.3
8  2020      2   Guido    HR   14.0
9  2020      2   Guido  Temp   37.6
10 2020      2     Sue    HR   45.0
11 2020      2     Sue  Temp   37.1
12 2021      1     Bob    HR   32.0
13 2021      1     Bob  Temp   37.0
14 2021      1   Guido    HR   37.0
15 2021      1   Guido  Temp   36.5
16 2021      1     Sue    HR   45.0
17 2021      1     Sue  Temp   39.1
18 2021      2     Bob    HR   69.0
19 2021      2     Bob  Temp   36.3
20 2021      2   Guido    HR   53.0
21 2021      2   Guido  Temp   36.1
22 2021      2     Sue    HR   44.0
23 2021      2     Sue  Temp   37.2

In [20]:
# Set the index back
health_data_flat.set_index(['year', 'visit', 'subject', 'type']).unstack(level=['subject', 'type'])

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2020 1      29.0  39.0  42.0  35.9  35.0  38.8
     2      30.0  36.3  14.0  37.6  45.0  37.1
2021 1      32.0  37.0  37.0  36.5  45.0  39.1
     2      69.0  36.3  53.0  36.1  44.0  37.2

### Swapping Index Levels

In [21]:
# Swap the order of the index levels
health_data.swaplevel('year', 'visit')

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
visit year                                    
1     2020  29.0  39.0  42.0  35.9  35.0  38.8
      2021  32.0  37.0  37.0  36.5  45.0  39.1
2     2020  30.0  36.3  14.0  37.6  45.0  37.1
      2021  69.0  36.3  53.0  36.1  44.0  37.2